In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")


In [3]:
from langchain_community.document_loaders import WebBaseLoader
loader =WebBaseLoader("https://docs.smith.langchain.com/evaluation/concepts")
loader

In [4]:
docs= loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/evaluation/concepts', 'title': 'Concepts | 🦜️🛠️ LangSmith', 'description': 'The pace of AI application development is often rate-limited by high-quality evaluations because there is a paradox of choice. Developers often wonder how to engineer their prompt or which LLM best balances accuracy, latency, and cost. High quality evaluations can help you rapidly answer these types of questions with confidence.', 'language': 'en'}, page_content='\n\n\n\n\nConcepts | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startObservabilityEvaluationConceptual GuideHow-to GuidesDatasetsEvaluationHuman feedbackTutorialsEvaluate your LLM applicationRAG EvaluationsBacktestingEvaluate an agentRunning SWE-bench with LangSmithPrompt EngineeringDeployment (LangGraph Cloud)AdministrationSelf-hostingReferenceEvaluationOn this pageConcepts\nThe pace of AI application development is often rate-limited by h

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap= 50)
text_chunks = splitter.split_documents(docs)
text_chunks

[Document(metadata={'source': 'https://docs.smith.langchain.com/evaluation/concepts', 'title': 'Concepts | 🦜️🛠️ LangSmith', 'description': 'The pace of AI application development is often rate-limited by high-quality evaluations because there is a paradox of choice. Developers often wonder how to engineer their prompt or which LLM best balances accuracy, latency, and cost. High quality evaluations can help you rapidly answer these types of questions with confidence.', 'language': 'en'}, page_content='Concepts | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/evaluation/concepts', 'title': 'Concepts | 🦜️🛠️ LangSmith', 'description': 'The pace of AI application development is often rate-limited by high-quality evaluations because there is a paradox of choice. Developers often wonder how to engineer their prompt or which LLM best balances accuracy, latency, and cost. High quality evaluations can help you rapidly answer these types of questions with confid

In [ ]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [ ]:
from langchain_community.vectorstores import FAISS
vector_store_db =FAISS.from_documents(text_chunks, embedding)


In [ ]:
vector_store_db

In [ ]:
query = "Once you have an application in production, you start getting valuable information: "
results= vector_store_db.similarity_search(query)
results[0].page_content


In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")
prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based on the context:
<context>

{context}

</context>

    """

)
document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

In [ ]:
from langchain_core.documents import Document
response_0 =document_chain.invoke({"input": "If your application is going well, you will likely get a lot of usage!",
                                 "context": [Document(page_content="If your application is going well, you will likely get a lot of usage!")]
                                 })


In [7]:
retriever =vector_store_db.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever, document_chain)
retrieval_chain

NameError: name 'vector_store_db' is not defined

In [ ]:
response =retrieval_chain.invoke({"input": "If your application is going well, you will likely get a lot of usage!"})
response

In [ ]:
response["answer"]

In [ ]:
response["context"]